In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [2]:
df = pd.read_csv("data/us_estdata.csv")
df.head()

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,JOBS_EDU_NOHS_ORIG,JOBS_EDU_HS_ORIG,JOBS_EDU_NOBACH_ORIG,JOBS_EDU_BACH_ORIG,HH_COST_ORIG_ADJ,PINCP_ORIG_ADJ,UNEMP_PCT_ORIG_ADJ,OWN_JOB_ORIG,TYPE_ORIG,CBSA_NAME_ORIG
0,0,P,2018GQ0103142,2,1,902,1,34,1013097,27,...,44574.0,75277.0,96405.0,139711.0,0.205419,32500.0,0.044023,0.0,0,367.0
1,1,P,2018HU1096416,9,1,10200,4,53,1013097,341,...,6449.0,14893.0,18922.0,14092.0,0.180426,32000.0,0.037921,0.0,1,495.0
2,2,P,2018HU0306186,7,2,1400,3,48,1013097,43,...,4016.0,6496.0,7050.0,4215.0,0.170502,30000.0,0.064784,0.0,0,101.0
3,3,P,2018HU0641097,7,1,1200,3,48,1013097,24,...,3882.0,7306.0,7491.0,4446.0,0.149895,24000.0,0.036782,0.0,2,95.0
4,4,P,2018HU1246748,5,1,7300,3,12,1013097,75,...,15006.0,31118.0,38494.0,34392.0,0.172667,30500.0,0.048288,0.0,1,150.0


In [3]:
cbsas_migpuma = pd.read_csv("data/migpuma_density.csv")
cbsas_migpuma["GISMATCH"] = cbsas_migpuma["GISMATCH"].astype(str).str.zfill(7)
cbsas_migpuma = cbsas_migpuma.set_index("GISMATCH")
cbsas_migpuma

,Unnamed: 0,STATEFIP,State,MIGPLAC,MIGPUMA,PWSTATE,PWPUMA,center,type,CBSA_name
GISMATCH,,,,,,,,,,
0600100,0,6,California,6,100,6,100,POINT (-2234513.518602708 322640.2378646344),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0600700,1,6,California,6,700,6,700,POINT (-2150235.360887794 534214.0203136775),Metro,"Chico, CA Metro Area"
0601100,2,6,California,6,1100,6,1100,POINT (-2216786.261006446 604917.4494366695),Micro,"Red Bluff, CA Micro Area"
0601300,3,6,California,6,1300,6,1300,POINT (-2229879.805136674 352695.1396070928),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0601500,4,6,California,6,1500,6,1500,POINT (-2096371.819538961 692307.6508446125),Metro,"Redding, CA Metro Area"
...,...,...,...,...,...,...,...,...,...,...
1304007,1000,13,Georgia,13,4007,13,4007,POINT (1095790.60191007 -326953.0717414605),T34,"Atlanta-Sandy Springs-Roswell, GA Metro Area"
5151000,1001,51,Virginia,51,51000,51,51000,POINT (1400332.713449827 105297.1155173389),Metro,"Roanoke, VA Metro Area"
5151001,1002,51,Virginia,51,51001,51,51001,POINT (1519180.668946094 191928.83683416),Metro,"Charlottesville, VA Metro Area"


In [4]:
df["ORIGIN"].value_counts()

603700     1997
2500390     937
1703400     870
400100      794
4804600     682
           ... 
2201600      11
1801700      11
2202100      10
1304400      10
4806900      10
Name: ORIGIN, Length: 975, dtype: int64

In [5]:
df["MED"] = np.where(df["NAICS"] == "MED", 1, 0)
df["MFG"] = np.where(df["NAICS"] == "MFG", 1, 0)
df["RET"] = np.where(df["NAICS"] == "RET", 1, 0)
df["EDU"] = np.where(df["NAICS"] == "EDU", 1, 0)
df["ADM"] = np.where(df["NAICS"] == "ADM", 1, 0)
df["FOD"] = np.where(df["NAICS"] == "FOD", 1, 0)
df["PRF"] = np.where(df["NAICS"] == "PRF", 1, 0)
df["TRN"] = np.where(df["NAICS"] == "TRN", 1, 0)
df["SRV"] = np.where(df["NAICS"] == "SRV", 1, 0)
df["FIN"] = np.where(df["NAICS"] == "FIN", 1, 0)
df["WHL"] = np.where(df["NAICS"] == "WHL", 1, 0)
df["AGR"] = np.where(df["NAICS"] == "AGR", 1, 0)
df["PUB"] = np.where(df["NAICS"] == "PUB", 1, 0)
df["INF"] = np.where(df["NAICS"] == "INF", 1, 0)
df["ENT"] = np.where(df["NAICS"] == "ENT", 1, 0)
df["REL"] = np.where(df["NAICS"] == "REL", 1, 0)
df["UTL"] = np.where(df["NAICS"] == "UTL", 1, 0)
df["EXT"] = np.where(df["NAICS"] == "EXT", 1, 0)
df["MNG"] = np.where(df["NAICS"] == "MNG", 1, 0)
df["CON"] = np.where(df["NAICS"] == "CON", 1, 0)

In [7]:
# clean up the database (Biogeme Database can only have numerical values)
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [8]:
df["CHOSEN"].value_counts()

102500     106
5500100     85
5310200     84
5500700     74
1700202     73
          ... 
4804619      7
2701404      7
5541001      6
2601703      5
3500806      4
Name: CHOSEN, Length: 2336, dtype: int64

In [9]:
# defining the chosen alterantive for each person explicitly (0 to 35, corresponding to staying and moving to one of the many PUMAs)
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 101): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])
df["CHOSEN"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [10]:
df["CHOSEN"].value_counts()

0    55261
1     8011
Name: CHOSEN, dtype: int64

In [11]:
df["IN_COLLEGE"] = np.where((df["SCHG"] == 15) | (df["SCHG"] == 16), 1, 0)
df["IN_COLLEGE"]

0        0
1        0
2        0
3        0
4        0
        ..
63267    0
63268    0
63269    0
63270    0
63271    0
Name: IN_COLLEGE, Length: 63272, dtype: int32

In [12]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [13]:
df["AGE_18_22"] = np.where(df["AGEP"] <= 22, 1, 0)
df["AGE_23_29"] = np.where((df["AGEP"] >= 23) & (df["AGEP"] <= 29), 1, 0)
df["AGE_30_39"] = np.where((df["AGEP"] >= 30) & (df["AGEP"] <= 39), 1, 0)
df["AGE_40_49"] = np.where((df["AGEP"] >= 40) & (df["AGEP"] <= 49), 1, 0)
df["AGE_50_64"] = np.where((df["AGEP"] >= 50) & (df["AGEP"] <= 64), 1, 0)

In [14]:
df["EDU_LESS_HIGH"] = np.where(df["SCHL"] <= 15, 1, 0)
df["EDU_HIGHONLY"] = np.where((df["SCHL"] >= 16) & (df["SCHL"] <= 17), 1, 0)
df["EDU_SOMECOLLEGE"] = np.where((df["SCHL"] >= 18) & (df["SCHL"] <= 20), 1, 0)
df["EDU_COLLEGE"] = np.where(df["SCHL"] >= 21, 1, 0)
df["EDU_NOCOLLEGE"] = np.where(df["EDU_COLLEGE"] == 0, 1, 0)

In [15]:
df["WOMAN_CHILD"] = np.where((df["PAOC"] >= 1) & (df["PAOC"] <= 3), 1, 0)
df["UNEMPLOYED"] = np.where(df["ESR"] == 3, 1, 0)

In [16]:
df["MALE"] = np.where(df["SEX"] == 1, 1, 0)
df["FEMALE"] = np.where(df["SEX"] == 0, 1, 0)

In [17]:
df["MARRIED"] = np.where(df["MAR"] == 1, 1, 0)

In [18]:
df["child_old"] = np.where(df["child"] == df["REC_CHILD"], 0, df["child"])
df["child_old"].value_counts()

0    44611
1    18661
Name: child_old, dtype: int64

In [19]:
df["REC_NO_MAR"] = np.where((df["MARHD"] == 1) | (df["MARHW"] == 1), 1, 0)
df["REC_NO_MAR"].value_counts()

0    62389
1      883
Name: REC_NO_MAR, dtype: int64

In [20]:
df["MARHM_new"] = np.where(df["MARHM"] == 2, 0, df["MARHM"])
df["MARHM_new"].value_counts()

0.0    62261
1.0     1011
Name: MARHM_new, dtype: int64

In [21]:
df["married_old"] = np.where((df["MARHM"] == df["MARRIED"]), 0, df["MARRIED"])
df["married_old"].value_counts()

1    33059
0    30213
Name: married_old, dtype: int64

In [22]:
df["MILITARY"] = np.where(df["MIL"] == 1, 1, 0)

In [23]:
# df["HH_COST"] = np.where(df["TEN"] == 3, df["GRNTP"], df["SMOCP"])

In [24]:
# update to debate still
df["AGR_EXT"] = np.where((df["AGR"] == 1) | (df["EXT"] == 1), 1, 0)
df["HIGH_ED"] = np.where((df["MED"] == 1) | (df["EDU"] == 1) | (df["PRF"] == 1) | (df["MED"] == 1) | (df["FIN"] == 1) | (df["INF"] == 1) | (df["MED"] == 1), 1, 0)
df["LICENSE"] = np.where((df["SRV"] == 1) | (df["REL"] == 1), 1, 0)
df["OTHER"] = np.where((df["AGR_EXT"] == 0) & (df["HIGH_ED"] == 0) & (df["LICENSE"] == 0), 1, 0)

In [25]:
df["MICRO_adj_ORIG"] = np.where((df["TYPE_ORIG"] == 3) | (df["TYPE_ORIG"] == 2), 1, 0)
df["METRO"] = np.where(df["TYPE_ORIG"] == 1, 1, 0)
df["T34"] = np.where(df["TYPE_ORIG"] == 0, 1, 0)

In [26]:
for i in range(1, 101):
    key = "ALT{0}_".format(i)
    df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
    df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
    df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\1465497578.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\1465497578.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\1465497578.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [27]:
df["UNEMP"] = np.where((df["ESR"] == 3) | (df["ESR"] == 6), 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\973991880.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["UNEMP"] = np.where((df["ESR"] == 3) | (df["ESR"] == 6), 1, 0)


In [28]:
df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
df["OTHER_FAMILY"] = np.where((df["HHT"] == 2) | (df["HHT"] == 3), 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\3935644085.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\3935644085.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_11812\3935644085.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [29]:
df.loc[df["CBSA_NAME_ORIG"] == -1, "CBSA_NAME_ORIG"] = -2

In [30]:
# df["INTERNAL"] = df["ORIGIN"] == df["CHOSEN"]

In [31]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df)

In [32]:
# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [33]:
c_move = Beta("c_move", 0, None, None, 0)

In [34]:
# Staying Choice Parameters to be Estimated
c_stay_married = Beta("c_stay_married", 0, None, None, 0)
c_stay_age_18_22 = Beta("c_stay_age_18_22", -0.673, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", -0.915, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", -0.509, None, None, 0)
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0.167, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0.519, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", -0.0884, None, None, 0)
c_stay_child = Beta("c_stay_child", -0.495, None, None, 0)
c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
c_stay = Beta("c_stay", 15, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
c_stay_college = Beta("c_stay_college", 0.67, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", -0.553, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", -0.38, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", -0.564, None, None, 0)
c_stay_mil = Beta("c_stay_mil", -1.17, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 0, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0, None, None, 0)

In [35]:
c_stay_income = Beta("c_stay_income", 0, 0, None, 0)
c_stay_hhinc = Beta("c_stay_hhinc", 0, 0, None, 0)
c_stay_hurent = Beta('c_stay_hurent', 0, None, 0, 0)
c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
c_stay_hhcost = Beta("c_stay_hhcost", 0, None, 0, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0, 0, None, 0)

In [36]:
c_stay_income = Beta("c_stay_income", 0, None, None, 0)
c_stay_hhinc = Beta("c_stay_hhinc", 0, None, None, 0)
c_stay_hurent = Beta('c_stay_hurent', 0, None, None, 0)
c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
c_stay_hhcost = Beta("c_stay_hhcost", 0, None, None, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, None, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0, None, None, 0)

In [37]:
# occupation-based coefficients

c_adm = Beta("c_adm", 0, None, None, 0)
c_agr = Beta("c_agr", 0, None, None, 0)
c_con = Beta("c_con", 0, None, None, 0)
c_edu = Beta("c_edu", 0, None, None, 0)
c_ent = Beta("c_ent", 0, None, None, 0)
c_ext = Beta("c_ext", 0, None, None, 0)
c_fin = Beta("c_fin", 0, None, None, 0)
c_fod = Beta("c_fod", 0, None, None, 0)
c_inf = Beta("c_inf", 0, None, None, 0)
c_med = Beta("c_med", 0, None, None, 0)
c_mfg = Beta("c_mfg", 0, None, None, 0)
c_mng = Beta("c_mng", 0, None, None, 0)
c_prf = Beta("c_prf", 0, None, None, 0)
c_pub = Beta("c_pub", 0, None, None, 0)
c_rel = Beta("c_rel", 0, None, None, 0)
c_ret = Beta("c_ret", 0, None, None, 0)
c_srv = Beta("c_srv", 0, None, None, 0)
c_trn = Beta("c_trn", 0, None, None, 0)
c_utl = Beta("c_utl", 0, None, None, 0)
c_whl = Beta("c_whl", 0, None, None, 0)

In [38]:
# defining the staying utility function
V0 = c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_edu_college * EDU_COLLEGE + c_stay_child * child_old + c_stay + c_stay_foreign * FOREIGN +  c_stay_dens * DENS_ORIG + c_stay_college * IN_COLLEGE + c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR + c_stay_rec_child * REC_CHILD + c_stay_mil * MILITARY + c_stay_hh_val * HH_MED_VAL_ORIG + married_old * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY + c_stay_unemp_rate * UNEMP_ORIG + c_stay_hhinc * HH_MED_INC_ORIG + c_stay_hurent * HH_MED_RENT_ORIG + c_stay_hhcost * HH_MED_RENT_ORIG * 12 / HH_MED_INC_ORIG + c_stay_vac * HU_VAC_ORIG / HU_TOT_ORIG

# c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_edu_college * EDU_COLLEGE + c_stay_child * child_old + c_stay + c_stay_foreign * FOREIGN +  c_stay_dens * DENS_ORIG + c_stay_college * IN_COLLEGE + c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR + c_stay_rec_child * REC_CHILD + c_stay_mil * MILITARY + c_stay_hhcost * (1 - UNEMP) * HH_COST_ORIG_ADJ + c_stay_vac * HU_VAC_ORIG / HU_TOT_ORIG + c_stay_unemp_rate * UNEMP_PCT_ORIG_ADJ + married_old * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY + c_stay_income * PINCP_ORIG_ADJ

In [39]:
# V1 = c_stay + c_stay_hhcost * HH_MED_RENT_ORIG / HH_MED_INC_ORIG * 12 + c_stay_unemp_rate * UNEMP_PCT_ORIG_ADJ + c_stay_income * AGG_INC_ORIG / TOT_POP_ORIG

In [40]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_destchoice_dist=Beta('c_destchoice_dist', -0.000000527, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', -0.352, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp", -0.371, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", -2.81, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0.0000664, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 3.49, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, 0, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 4.18, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
c_destchoice_entscore = Beta("c_destchoice_entscore", 1.31, None, None, 0)
c_destchoice_pincp = Beta("c_destchoice_pincp", 0, None, None, 0)

In [41]:
c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0.708, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0.761, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0.517, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 1.26, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 1.28, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", -0.0517, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0.941, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 1.82, None, None, 0)

In [42]:
c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 5.47, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 1.15, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 8.01, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 1.61, None, None, 0)

In [43]:
c_destchoice_samestate = Beta("c_destchoice_samestate", 0, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 2.04, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 1.41, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", -0.0000333, None, None, 0)

In [44]:
# birthstate > cbsa > state

In [45]:
# defining the utility functions for each of the moving PUMA alternatives
# defined using the exec to parse a string to save space
# can also use a loop to print out the statements and then copy/paste them to run
# can also just write each one manually
for i in range(100):
    num = i + 1
    initialization = "V{0} = log(ALT{0}_POP) + (CBSA_NAME_ORIG != ALT{0}_CBSA) * (c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1)) + (CBSA_NAME_ORIG == ALT{0}_CBSA) * c_destchoice_cbsa_dist * ALT{0}_DIST + \
    c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * ALT{0}_35_64 + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_entscore * ALT{0}_ENT / AGEP / (ALT{0}_EMP + 1) + c_destchoice_unemp_rate * ALT{0}_UNEMP + \
    (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
    (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
    (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
    (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER) * ALT{0}_OWN_JOB / (ALT{0}_EMP + 1) + \
    c_destchoice_birthstate * (POBP == ALT{0}_STATE) * (CBSA_NAME_ORIG != ALT{0}_CBSA) + c_destchoice_samecbsa * (CBSA_NAME_ORIG == ALT{0}_CBSA)".format(num)
    exec(initialization)
print(V100)

# full model specification (takes a bit longer to run):
# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_entscore * ALT{0}_ENT / AGEP / ALT{0}_EMP + c_destchoice_unemp * ALT{0}_UNEMP + c_destchoice_pincp * ALT{0}_PINCP + \
# (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
# (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
# (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
# (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER) * ALT{0}_OWN_JOB / ALT{0}_EMP".format(num)

# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_adm * ADM * ALT{0}_ADM/ALT{0}_EMP + c_agr * AGR * ALT{0}_AGR/ALT{0}_EMP + c_con * CON * ALT{0}_CON/ALT{0}_EMP + c_edu * EDU * ALT{0}_EDU/ALT{0}_EMP + c_ent * ENT * ALT{0}_ENT/ALT{0}_EMP + c_ext * EXT * ALT{0}_EXT/ALT{0}_EMP + c_fin * FIN * ALT{0}_FIN/ALT{0}_EMP + c_fod * FOD * ALT{0}_FOD/ALT{0}_EMP + c_inf * INF * ALT{0}_INF/ALT{0}_EMP + c_med * MED * ALT{0}_MED/ALT{0}_EMP + c_mfg * MFG * ALT{0}_MFG/ALT{0}_EMP + c_mng * MNG * ALT{0}_MNG/ALT{0}_EMP + c_prf * PRF * ALT{0}_PRF/ALT{0}_EMP + c_pub * PUB * ALT{0}_PUB/ALT{0}_EMP + c_rel * REL * ALT{0}_REL/ALT{0}_EMP + c_ret * RET * ALT{0}_RET/ALT{0}_EMP + c_srv * SRV * ALT{0}_SRV/ALT{0}_EMP + c_trn * TRN * ALT{0}_TRN/ALT{0}_EMP + c_utl * UTL * ALT{0}_UTL/ALT{0}_EMP + c_whl * WHL * ALT{0}_WHL/ALT{0}_EMP"

# for the fields already in the Biogeme db.Database, can explicitly refer to them; also used a few references to other databases using .loc and fields in the Biogeme database

((((((((((((((((log(ALT100_POP) + ((CBSA_NAME_ORIG != ALT100_CBSA) * ((c_destchoice_dist(-5.27e-07) * ALT100_DIST) + (c_destchoice_logdist(-0.352) * log((ALT100_DIST + `1`)))))) + (((CBSA_NAME_ORIG == ALT100_CBSA) * c_destchoice_cbsa_dist(-3.33e-05)) * ALT100_DIST)) + (c_destchoice_hhcost(-2.81) * ALT100_HH_COST)) + ((c_destchoice_college(6.64e-05) * IN_COLLEGE) * ALT100_COLLEGE)) + (((c_destchoice_foreign(0) * FOREIGN) * ALT100_FOREIGN) / ALT100_POP)) + ((c_destchoice_age_18_34(3.49) * ALT100_18_34) / ALT100_POP)) + (c_destchoice_age_35_64(0) * ALT100_35_64)) + ((c_destchoice_age_over_65(4.18) * ALT100_65) / ALT100_POP)) + (((c_destchoice_entscore(1.31) * ALT100_ENT) / AGEP) / (ALT100_EMP + `1`))) + (c_destchoice_unemp(-0.371) * ALT100_UNEMP)) + ((((c_destchoice_T34_T34(0) * T34) + (c_destchoice_Metro_T34(0.517) * METRO)) + (c_destchoice_Micro_T34(-0.0517) * MICRO_adj_ORIG)) * ALT100_T34)) + ((((c_destchoice_T34_Metro(0.708) * T34) + (c_destchoice_Metro_Metro(1.26) * METRO)) + (c_dest

In [46]:
W0 = PWGTP * 1

In [47]:
utilities = {}
for i in range(101):
    init = "utilities[{0}] = V{0}".format(i)
    exec(init)

In [48]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-06-29]\n'

In [49]:
import biogeme.optimization as opt

In [51]:
# Associate utility functions with the numbering of alternatives (corresponds to the CHOSEN field created earlier)
V = utilities

# Associate the availability conditions with the alternatives
# for this model, all migrants had all alternatives theoretically available so all are equal to 1 (available)
# if individual people had different availability for alterantives, could pass in a column of the dataframe to account for that availability
av = {}
for i in range(0, 101):
    av[i] = 1

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, av, CHOSEN)

formulas = {"loglike": logprob, "weight": W0}

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, suggestScales=False)
biogeme.modelName = 'us_full'

biogeme.saveIterations = False

# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[01:15:43] < General >   Remove 2698 unused variables from the database as only 1736 are used.
[01:17:05] < Detailed >  Log likelihood (N = 63272):  -40126.18
[01:17:05] < Detailed >  ** Optimization: Newton with trust region for simple bounds
[01:36:04] < General >   Log likelihood (N = 63272):  -40126.18 Gradient norm:      5e+09 Hessian norm:       1e+16 
[01:38:54] < Detailed >  Log likelihood (N = 63272):  -32991.49
[01:57:12] < General >   Log likelihood (N = 63272):  -32991.49 Gradient norm:      1e+09 Hessian norm:       6e+15 
[01:57:12] < Detailed >  1 f= 0.5214232 projected rel. grad.=1.4e+04 rel. change=     1 delta=     2 rho=   1.1 ++
[02:00:06] < Detailed >  Log likelihood (N = 63272):  -32597.19
[02:18:11] < General >   Log likelihood (N = 63272):  -32597.19 Gradient norm:      2e+08 Hessian norm:       4e+15 
[02:18:11] < Detailed >  2 f= 0.5151914 projected rel. grad.=2.4e+03 rel. change=  0.67 delta=     4 rho=   1.1 ++
[02:20:57] < Detailed >  Log likelihood (N = 63

In [ ]:
# utilities = {0: V0, 1: V1}

# # Associate utility functions with the numbering of alternatives (corresponds to the CHOSEN field created earlier)
# V = utilities

# # Associate the availability conditions with the alternatives
# # for this model, all migrants had all alternatives theoretically available so all are equal to 1 (available)
# # if individual people had different availability for alterantives, could pass in a column of the dataframe to account for that availability
# av = {}
# for i in range(0, 2):
#     av[i] = 1

# # Definition of the model. This is the contribution of each
# # observation to the log likelihood function.
# # estimating the CHOSEN field
# logprob = models.loglogit(V, av, STAY)


# # Create the Biogeme object
# biogeme = bio.BIOGEME(database, logprob)
# biogeme.modelName = 'us_full_basic'

# # Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
# biogeme.calculateNullLoglikelihood(av)

# # Estimate the parameters
# results = biogeme.estimate()

# # Get the results in a pandas table
# pandasResults = results.getEstimatedParameters()
# print(pandasResults)